### P2 - Weather Data (APIs) - Part 1

GROUP #14 DETAILS<br>
<br>
 1	Amita Singh                 219485309	amita15@schulich.yorku.ca<br>
 2	Saima Rafi	                216562753	saima20@schulich.yorku.ca<br>
 3	Sreekanth Potlabathini 	    220088993	sriku241@schulich.yorku.ca<br>
 4	Abdul Ghafar Qasemi	        220067401	agqasemi@schulich.yorku.ca<br>  
 

--------------------------------------------------------------------------------------------------------------

For this project you will work on three data pipelines<br>
<br>
A common problem in Logistics is the optimization of delivery routes using distance and weather data. For this assignment you will need to identify an API that can provide:<br>
<br>
Historical Weather data<br>( no free version found)
### Current Weather<br>
### Weather Forecasts<br> 
file contains only highlighted portion<br> 
Weather Alerts (you will need to work with the data available here: https://dd.weather.gc.ca/Links to an external site.)<br> 
Most APIs from big providers provide (limited) free requests that should allow you to pull a sample dataset without having to pay for a subscription<br>
<br>
Create an uploads schema in your own database within our Postgres server (do not use the mban_db database)<br>
Prepare a data ingestion pipeline for each data source. The data ingestion pipeline must be designed to be scheduled (i.e. it should take the current date as an input so it can run every day and update your tables)<br>
Each data ingestion pipeline must include data cleaning and feature engineering<br>
Create new tables in the uploads schema to capture the weather data.<br>

--------------------------------------------------------------------------------------------------------------

# For Weather Forcast

In [38]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import requests
import sys
import datetime
from datetime import datetime
import json

In [39]:
lat = '43.8143'
long = '-79.4240'
limit = 1500
offset = 0
appid = '69b9eac66ab7d3985df833d77d77bc86'

In [40]:

api_url_2 = 'https://api.openweathermap.org/data/2.5/forecast?lat={}&lon={}&limit={}&offset={}&appid={}'.format(lat,long, limit, offset, appid)
print(api_url_2)

https://api.openweathermap.org/data/2.5/forecast?lat=43.8143&lon=-79.4240&limit=1500&offset=0&appid=69b9eac66ab7d3985df833d77d77bc86


In [41]:
api_response_2 = requests.get(api_url_2)
api_response_2

<Response [200]>

In [42]:
data_f = api_response_2.json()
data_f

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1691971200,
   'main': {'temp': 295.66,
    'feels_like': 295.64,
    'temp_min': 292.45,
    'temp_max': 295.66,
    'pressure': 1014,
    'sea_level': 1014,
    'grnd_level': 995,
    'humidity': 64,
    'temp_kf': 3.21},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': {'all': 99},
   'wind': {'speed': 4.26, 'deg': 333, 'gust': 8.01},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-08-14 00:00:00'},
  {'dt': 1691982000,
   'main': {'temp': 292.32,
    'feels_like': 292.15,
    'temp_min': 289.84,
    'temp_max': 292.32,
    'pressure': 1015,
    'sea_level': 1015,
    'grnd_level': 996,
    'humidity': 71,
    'temp_kf': 2.48},
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04n'}],
   'clouds': {'all': 91},
   'wind': {'speed': 3.26, 'deg': 321, 'gust': 8.53},
   '

In [43]:
print(data_f.keys())

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])


In [44]:
forcast_df = pd.DataFrame(data_f['list'])

In [45]:
current_date = datetime.now().date()

In [46]:
main = ['temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'temp_kf']
for column in main:
    forcast_df [f'main_{column}'] = forcast_df ['main'].apply(lambda x: x[column])

w_c = ['id', 'main', 'description', 'icon']
for column in w_c:
    forcast_df [f'weather_{column}'] = forcast_df ['weather'].apply(lambda x: x[0][column])

forcast_df .drop(['main', 'weather'], axis=1, inplace=True)

forcast_df .sample(10)

,dt,clouds,wind,visibility,pop,sys,dt_txt,rain,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_temp_kf,weather_id,weather_main,weather_description,weather_icon
28,1692273600,{'all': 67},"{'speed': 3.25, 'deg': 189, 'gust': 9.21}",10000,0.00,{'pod': 'd'},2023-08-17 12:00:00,NaN,291.65,291.57,291.65,291.65,1010,77,0.00,803,Clouds,broken clouds,04d
5,1692025200,{'all': 88},"{'speed': 1.63, 'deg': 99, 'gust': 1.77}",10000,0.00,{'pod': 'd'},2023-08-14 15:00:00,NaN,294.19,293.73,294.19,294.19,1017,53,0.00,804,Clouds,overcast clouds,04d
29,1692284400,{'all': 59},"{'speed': 5.94, 'deg': 206, 'gust': 9.98}",10000,0.00,{'pod': 'd'},2023-08-17 15:00:00,NaN,296.95,296.93,296.95,296.95,1008,59,0.00,803,Clouds,broken clouds,04d
13,1692111600,{'all': 100},"{'speed': 4.47, 'deg': 56, 'gust': 10.09}",9739,1.00,{'pod': 'd'},2023-08-15 15:00:00,{'3h': 2.34},289.26,289.35,289.26,289.26,1009,93,0.00,500,Rain,light rain,10d
14,1692122400,{'all': 100},"{'speed': 4.54, 'deg': 47, 'gust': 9.45}",10000,1.00,{'pod': 'd'},2023-08-15 18:00:00,{'3h': 1.41},289.46,289.55,289.46,289.46,1009,92,0.00,500,Rain,light rain,10d
20,1692187200,{'all': 58},"{'speed': 2.07, 'deg': 337, 'gust': 5.08}",10000,0.00,{'pod': 'd'},2023-08-16 12:00:00,NaN,289.94,289.92,289.94,289.94,1014,86,0.00,803,Clouds,broken clouds,04d
12,1692100800,{'all': 100},"{'speed': 5.25, 'deg': 69, 'gust': 11.71}",10000,0.89,{'pod': 'd'},2023-08-15 12:00:00,{'3h': 0.57},289.74,289.80,289.74,289.74,1008,90,0.00,500,Rain,light rain,10d
16,1692144000,{'all': 100},"{'speed': 2.89, 'deg': 4, 'gust': 6.6}",10000,1.00,{'pod': 'd'},2023-08-16 00:00:00,{'3h': 1.22},289.02,289.12,289.02,289.02,1011,94,0.00,500,Rain,light rain,10d
3,1692003600,{'all': 71},"{'speed': 1.73, 'deg': 340, 'gust': 3.24}",10000,0.00,{'pod': 'n'},2023-08-14 09:00:00,NaN,287.54,287.23,287.54,287.54,1016,84,0.00,803,Clouds,broken clouds,04n
0,1691971200,{'all': 99},"{'speed': 4.26, 'deg': 333, 'gust': 8.01}",10000,0.00,{'pod': 'd'},2023-08-14 00:00:00,NaN,295.66,295.64,292.45,295.66,1014,64,3.21,804,Clouds,overcast clouds,04d


In [47]:
forcast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dt                   40 non-null     int64  
 1   clouds               40 non-null     object 
 2   wind                 40 non-null     object 
 3   visibility           40 non-null     int64  
 4   pop                  40 non-null     float64
 5   sys                  40 non-null     object 
 6   dt_txt               40 non-null     object 
 7   rain                 9 non-null      object 
 8   main_temp            40 non-null     float64
 9   main_feels_like      40 non-null     float64
 10  main_temp_min        40 non-null     float64
 11  main_temp_max        40 non-null     float64
 12  main_pressure        40 non-null     int64  
 13  main_humidity        40 non-null     int64  
 14  main_temp_kf         40 non-null     float64
 15  weather_id           40 non-null     int64

In [48]:
forcast_df ['dt'] = pd.to_datetime(forcast_df ['dt_txt'])

In [49]:
forcast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   dt                   40 non-null     datetime64[ns]
 1   clouds               40 non-null     object        
 2   wind                 40 non-null     object        
 3   visibility           40 non-null     int64         
 4   pop                  40 non-null     float64       
 5   sys                  40 non-null     object        
 6   dt_txt               40 non-null     object        
 7   rain                 9 non-null      object        
 8   main_temp            40 non-null     float64       
 9   main_feels_like      40 non-null     float64       
 10  main_temp_min        40 non-null     float64       
 11  main_temp_max        40 non-null     float64       
 12  main_pressure        40 non-null     int64         
 13  main_humidity        40 non-null     

# Push Weather Forcast to postgres server

In [50]:
db_secret= {
    'drivername': 'postgresql+psycopg2',
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432',
   'username': 'sriku241',
   'password': '2023!Schulich',
   'database': 'sriku241_db'
}

In [51]:
db_connection_url= sa.engine.URL.create(

drivername= db_secret['drivername'],
username= db_secret['username'],
password= db_secret['password'],
host= db_secret['host'],
port= db_secret['port'],
database= db_secret['database']
)

In [52]:
engine= sa.create_engine(db_connection_url)

In [53]:
create_schema_query = sa.text("CREATE SCHEMA IF NOT EXISTS p2_group_assignment")
with engine.connect() as connection:
    connection.execute(create_schema_query)
    

In [54]:
forcast_df .columns

Index(['dt', 'clouds', 'wind', 'visibility', 'pop', 'sys', 'dt_txt', 'rain',
       'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max',
       'main_pressure', 'main_humidity', 'main_temp_kf', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')

In [55]:
forcast_weather = forcast_df [['dt','visibility', 'pop', 'rain', 'sys',
       'main_temp', 'main_feels_like', 'main_temp_min', 'main_temp_max',
       'main_pressure', 'main_humidity', 'main_temp_kf',
       'weather_main', 'weather_description']]

In [56]:
forcast_weather.dtypes

dt                     datetime64[ns]
visibility                      int64
pop                           float64
rain                           object
sys                            object
main_temp                     float64
main_feels_like               float64
main_temp_min                 float64
main_temp_max                 float64
main_pressure                   int64
main_humidity                   int64
main_temp_kf                  float64
weather_main                   object
weather_description            object
dtype: object

In [57]:
forcast_weather['rain_3h'] = forcast_weather['rain'].apply(lambda x: x['3h'] if isinstance(x, dict) and '3h' in x else np.nan)
forcast_weather.drop(columns=['rain'], inplace=True)
forcast_weather.sample(5)

C:\Users\srika\AppData\Local\Temp\ipykernel_11360\458232954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcast_weather['rain_3h'] = forcast_weather['rain'].apply(lambda x: x['3h'] if isinstance(x, dict) and '3h' in x else np.nan)
C:\Users\srika\AppData\Local\Temp\ipykernel_11360\458232954.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcast_weather.drop(columns=['rain'], inplace=True)


,dt,visibility,pop,sys,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,main_temp_kf,weather_main,weather_description,rain_3h
33,2023-08-18 03:00:00,10000,0.74,{'pod': 'n'},289.14,289.04,289.14,289.14,1006,86,0.00,Rain,light rain,0.64
37,2023-08-18 15:00:00,10000,0.08,{'pod': 'd'},290.28,289.69,290.28,290.28,1013,63,0.00,Clouds,overcast clouds,NaN
6,2023-08-14 18:00:00,10000,0.00,{'pod': 'd'},296.51,296.10,296.51,296.51,1015,46,0.00,Clouds,overcast clouds,NaN
3,2023-08-14 09:00:00,10000,0.00,{'pod': 'n'},287.54,287.23,287.54,287.54,1016,84,0.00,Clouds,broken clouds,NaN
1,2023-08-14 03:00:00,10000,0.00,{'pod': 'n'},292.32,292.15,289.84,292.32,1015,71,2.48,Clouds,overcast clouds,NaN


In [58]:
forcast_weather['sys'] = forcast_weather['sys'].apply(json.dumps)

C:\Users\srika\AppData\Local\Temp\ipykernel_11360\4160365095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forcast_weather['sys'] = forcast_weather['sys'].apply(json.dumps)


In [59]:
import warnings
warnings.filterwarnings("ignore")

In [60]:
forcast_weather.to_sql(
    name='forcast_weather',
    schema='p2_group_assignment',
    con=engine,
    if_exists='replace',  
    index=False,
    dtype={
        'dt': sa.types.DateTime,
        'clouds': sa.types.VARCHAR(255),  
        'wind': sa.types.VARCHAR(255),
        'visibility': sa.types.Integer,
        'pop': sa.types.DECIMAL(10,2),
        'sys': sa.types.VARCHAR(255),
        'rain_3h': sa.types.DECIMAL(10,2),  
        'main_temp': sa.types.DECIMAL(10,2),
        'main_feels_like': sa.types.DECIMAL(10,2),
        'main_temp_min': sa.types.DECIMAL(10,2),
        'main_temp_max': sa.types.DECIMAL(10,2),
        'main_pressure': sa.types.Integer,
        'main_humidity': sa.types.DECIMAL(10,2),
        'main_temp_kf': sa.types.DECIMAL(10,2),
        'weather_main': sa.types.VARCHAR(255),
        'weather_description': sa.types.VARCHAR(255)
    }
)


40

# For Current Waether

In [61]:
api_url_1 = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&limit={}&offset={}&appid={}'.format(lat,long, limit, offset, appid)
print(api_url_1)

https://api.openweathermap.org/data/2.5/weather?lat=43.8143&lon=-79.4240&limit=1500&offset=0&appid=69b9eac66ab7d3985df833d77d77bc86


In [62]:
api_response_1 = requests.get(api_url_1)
api_response_1

<Response [200]>

In [63]:
data_w = api_response_1.json()
data_w

{'coord': {'lon': -79.424, 'lat': 43.8143},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 297.27,
  'feels_like': 297.38,
  'temp_min': 295.93,
  'temp_max': 299.81,
  'pressure': 1013,
  'humidity': 63},
 'visibility': 10000,
 'wind': {'speed': 1.79, 'deg': 338, 'gust': 4.02},
 'clouds': {'all': 100},
 'dt': 1691962028,
 'sys': {'type': 2,
  'id': 2036018,
  'country': 'CA',
  'sunrise': 1691921968,
  'sunset': 1691972755},
 'timezone': -14400,
 'id': 6165683,
 'name': 'Thornhill',
 'cod': 200}

In [64]:
print(data_w.keys())

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])


In [65]:
current_weather = pd.json_normalize(data_w)
current_weather

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1691962028,-14400,6165683,Thornhill,200,-79.424,43.8143,...,63,1.79,338,4.02,100,2,2036018,CA,1691921968,1691972755


In [66]:
current_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   weather          1 non-null      object 
 1   base             1 non-null      object 
 2   visibility       1 non-null      int64  
 3   dt               1 non-null      int64  
 4   timezone         1 non-null      int64  
 5   id               1 non-null      int64  
 6   name             1 non-null      object 
 7   cod              1 non-null      int64  
 8   coord.lon        1 non-null      float64
 9   coord.lat        1 non-null      float64
 10  main.temp        1 non-null      float64
 11  main.feels_like  1 non-null      float64
 12  main.temp_min    1 non-null      float64
 13  main.temp_max    1 non-null      float64
 14  main.pressure    1 non-null      int64  
 15  main.humidity    1 non-null      int64  
 16  wind.speed       1 non-null      float64
 17  wind.deg         1 n

In [67]:
current_weather['dt'] = pd.to_datetime(current_weather['dt'], unit='s')
current_weather['sys.sunrise'] = pd.to_datetime(current_weather['sys.sunrise'], unit='s')
current_weather['sys.sunset'] = pd.to_datetime(current_weather['sys.sunset'], unit='s')
current_weather

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,2023-08-13 21:27:08,-14400,6165683,Thornhill,200,-79.424,43.8143,...,63,1.79,338,4.02,100,2,2036018,CA,2023-08-13 10:19:28,2023-08-14 00:25:55


In [68]:
current_date = datetime.now().date()

In [69]:
current_weather['current_date'] = current_date

In [70]:
cols = current_weather.columns.tolist()
cols = ['current_date'] + [col for col in cols if col != 'current_date']
current_weather = current_weather[cols]
current_weather

,current_date,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,...,main.humidity,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,2023-08-13,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,2023-08-13 21:27:08,-14400,6165683,Thornhill,200,-79.424,...,63,1.79,338,4.02,100,2,2036018,CA,2023-08-13 10:19:28,2023-08-14 00:25:55


In [71]:
current_weather['weather_id'] = current_weather['weather'].apply(lambda x: x[0]['id'])
current_weather['weather_main'] = current_weather['weather'].apply(lambda x: x[0]['main'])
current_weather['weather_description'] = current_weather['weather'].apply(lambda x: x[0]['description'])
current_weather.drop(columns=['weather'], inplace=True)


In [72]:
current_weather.columns

Index(['current_date', 'base', 'visibility', 'dt', 'timezone', 'id', 'name',
       'cod', 'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'wind.speed', 'wind.deg', 'wind.gust', 'clouds.all', 'sys.type',
       'sys.id', 'sys.country', 'sys.sunrise', 'sys.sunset', 'weather_id',
       'weather_main', 'weather_description'],
      dtype='object')

# Push Current Weather to Postgres database

####  Ingest the data into your own database in our PostgreSQL server<br>

In [73]:
current_weather.to_sql(
    name = 'current_weather',
    schema= 'p2_group_assignment',
    con = engine,
    if_exists = 'replace',
    index = False,
    dtype = {
        'base' : sa.types.VARCHAR(255),
        'visibility' : sa.Integer,
        'dt': sa.types.DateTime,
        'timezone': sa.types.Integer,
        'id': sa.types.Integer,
        'name': sa.types.VARCHAR(255),
        'cod': sa.types.Integer,
        'coord.lon': sa.types.DECIMAL(10,2),
        'coord.lat': sa.types.DECIMAL(10,2),
        'weather.id': sa.types.Integer,
        'weather.main': sa.types.VARCHAR(255),
        'weather.description': sa.types.VARCHAR(60),
        'weather.icon': sa.types.VARCHAR(255),
        'main.temp': sa.types.DECIMAL(10,2),
        'main.feels_like': sa.types.DECIMAL(10,2),
        'main.temp_min': sa.types.DECIMAL(10,2),
        'main.temp_max': sa.types.DECIMAL(10,2),
        'main.pressure': sa.types.Integer,
        'main.humidity': sa.types.Integer,
        'wind.speed': sa.types.DECIMAL(10,2),
        'wind.deg': sa.types.Integer,
        'wind.gust': sa.types.DECIMAL(10,2),
        'clouds.all': sa.types.Integer,
        'sys.type': sa.types.Integer,
        'sys.id': sa.types.Integer,
        'sys.country': sa.types.VARCHAR(255),
        'sys.sunrise': sa.types.DateTime,
        'sys.sunset': sa.types.DateTime,
        'weather_id': sa.types.Integer,
        'weather_main': sa.types.VARCHAR(255),
        'weather_description': sa.types.VARCHAR(255)
        }
)

1

In [74]:
warnings.resetwarnings()